In [ ]:
%matplotlib notebook
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func , inspect, and_
from flask import Flask, jsonify
engine = create_engine("sqlite:///hawaii.sqlite")
 # reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
 # We can view all of the classes that automap found
Base.classes.keys()
 # Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station
 # Create our session (link) from Python to the DB
session = Session(engine)
app = Flask(__name__)

@app.route("/api/v1.0/precipitation")
def list_precipitation():
    # Calculate the date 1 year ago from today
    year_ago = dt.date.today() - dt.timedelta(days=365)
    # Perform a query to retrieve the data and precipitation scores
    results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= year_ago).all()
    dic = {}
    for d, p in results:
        dic[d] = p
    print(dic)
    return jsonify(dic)

@app.route("/api/v1.0/stations")
def list_stations():
    stations = session.query(Station.station).group_by(Station.station).all()
    print(stations)
    return jsonify(stations)
@app.route("/api/v1.0/tobs")
def list_tobs():
    year_ago = dt.date.today() - dt.timedelta(days=365)
    temperature = session.query(Measurement.tobs).\
    filter(and_(Measurement.station == 'USC00519281', Measurement.date >= year_ago)).all()
    return jsonify(temperature)
@app.route("/api/v1.0/<start>")
def start_date(start):
    print(start)
    result = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs))\
           .filter(Measurement.date >= start).first()
    print(result)
    return jsonify(result)

@app.route("/api/v1.0/<start>/<end>")
def start_end_date(start,end):
    print(start)
    print(end)
    start_end = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs))\
           .filter(and_(Measurement.date >= start,Measurement.date <= end)).all()
    print(start_end)
    return jsonify(start_end)

if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Aug/2018 18:53:57] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [05/Aug/2018 18:54:21] "GET /api/v1.0/precipitation HTTP/1.1" 200 -


{'2017-08-05': None, '2017-08-06': None, '2017-08-07': 0.05, '2017-08-08': 0.34, '2017-08-09': 0.15, '2017-08-10': 0.07, '2017-08-11': None, '2017-08-12': 0.14, '2017-08-13': None, '2017-08-14': 0.22, '2017-08-15': 0.42, '2017-08-18': None, '2017-08-19': 0.09, '2017-08-20': None, '2017-08-21': 0.56, '2017-08-22': 0.5, '2017-08-23': 0.45, '2017-08-16': 0.42, '2017-08-17': 0.13}
